In [1]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import RandomizedSearchCV
from random import randrange
from math import sqrt

class LVQClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, n_codebooks=10, learn_rate=0.1, n_epochs=50):
        self.n_codebooks = n_codebooks
        self.learn_rate = learn_rate
        self.n_epochs = n_epochs
        self.codebooks = None
    
    def euclidean_distance(self, row1, row2):
        return sqrt(sum((row1[i] - row2[i])**2 for i in range(len(row1)-1)))
    
    def get_best_matching_unit(self, test_row):
        distances = [(cb, self.euclidean_distance(cb, test_row)) for cb in self.codebooks]
        return min(distances, key=lambda x: x[1])[0]
    
    def train_codebooks(self, X_train, Y_train):
        train = np.column_stack((X_train, Y_train))
        self.codebooks = [train[randrange(len(train))].tolist() for _ in range(self.n_codebooks)]
        
        for epoch in range(self.n_epochs):
            rate = self.learn_rate * (1.0 - (epoch / float(self.n_epochs)))
            for row in train:
                bmu = self.get_best_matching_unit(row)
                for i in range(len(row)-1):
                    error = row[i] - bmu[i]
                    bmu[i] += rate * error if bmu[-1] == row[-1] else -rate * error
    
    def fit(self, X_train, Y_train):
        self.train_codebooks(X_train, Y_train)
        return self
    
    def predict(self, X_test):
        return np.array([self.get_best_matching_unit(row)[-1] for row in X_test])
    
    def score(self, X_test, Y_test):
        predictions = self.predict(X_test)
        return np.mean(predictions == Y_test)


# Fase 4: Busca de Parâmetros

Nesta fase iremos aprofundar a busca de hiperparamewtros, variabndo os parametros ca cada modelos, e compaerandos seus resultados

O primeiro passo que iremos fazer é importar as bibliotecas principais, e estabelecer o ambiente de execução.


In [2]:
# Bibliotecas Externas
import os
import warnings
import pandas as pd
import numpy as np

# Ignorando Warnings
warnings.filterwarnings("ignore")

# Bibliotecas Locais
os.chdir("../..")
from lib import plots, runner

# Definido Variáveis globais
N_SPLITS = 5
RANDOM_STATE = 51
LEARN_RATE = 0.3
N_EPOCHS = 50
N_CODEBOOKS = 15

## 0. Importando os Dados

Nosso primero é coletar os dados que preparamos na etapa anterior, para verificar como esses dados se comportam

In [3]:
# Conjunto de treino
X_train = pd.read_csv('./data/processed/X_train.csv')
Y_train = pd.read_csv('./data/processed/Y_train.csv')

Y_train['class'] = Y_train['class'].apply(lambda val: 1 if val == 'UP' else 0)

# Conjunto de validação
X_val = pd.read_csv('./data/processed/X_val.csv')
Y_val = pd.read_csv('./data/processed/Y_val.csv')


Y_val['class'] = Y_val['class'].apply(lambda val: 1 if val == 'UP' else 0)

# Conjunto de teste
X_test = pd.read_csv('./data/processed/X_test.csv')
Y_test = pd.read_csv('./data/processed/Y_test.csv')

Y_test['class'] = Y_test['class'].apply(lambda val: 1 if val == 'UP' else 0)

## 1. Algoritmos de Machine Learning para Classificação

### LVQ (Learning Vector Quantization)

#### Juntando os dados

In [4]:
train_data = np.hstack((X_train.values, Y_train.values))
test_data = np.hstack((X_test.values, Y_test.values))

#### Avaliação do modelo

- n_codebooks: Define quantos protótipos serão usados no LVQ. Um número maior pode melhorar a capacidade de representação, mas também pode aumentar o tempo de treinamento.

- learn_rate: Controla a taxa de ajuste dos protótipos durante o treinamento. Valores menores podem levar a uma convergência mais estável, enquanto valores maiores podem acelerar o aprendizado, mas com risco de oscilação.

- n_epochs: Número de iterações sobre o conjunto de treinamento. Um número maior pode melhorar a generalização, mas pode levar a overfitting se muito alto.

In [ ]:
# Criar o modelo LVQ base
lvq_model = LVQClassifier()

In [5]:
param_distributions_lvq = {
    'n_codebooks': [5, 10, 15, 20, 30],  # Número de protótipos
    'learn_rate': [0.01, 0.05, 0.1, 0.3, 0.5],  # Taxa de aprendizado inicial
    'n_epochs': [10, 30, 50, 100],  # Número de épocas (iterações de treinamento)
}

In [ ]:
"""

# Criar o RandomizedSearchCV
search = RandomizedSearchCV(
    estimator=lvq_model,
    param_distributions=param_distributions_lvq,
    n_iter=50,  # Número de combinações testadas
    cv=5,  # Validação cruzada com 5 folds
    scoring='accuracy',  # Métrica de avaliação
    random_state=42,
    n_jobs=-1  # Paralelizar o processo
)

# Executar a busca pelos melhores hiperparâmetros
search.fit(X_train, Y_train)"""

KeyboardInterrupt: 

In [ ]:
"""# Mostrar os melhores parâmetros encontrados
print("Melhores parâmetros:", search.best_params_)
print("Melhor acurácia:", search.best_score_)"""

In [ ]:
df_lvq, model_lvq, results_lvq, df_iter = runner.new_search_params(
    model=lvq_model,
    params=param_distributions_lvq,
    X_train=X_train,
    Y_train=Y_train,
    model_name="LVQ",
    max_combinations=20,
    stop_iter=10,
    load_model=False,
    save=True
)

Iniciando busca por hiperparâmetros...


In [ ]:
df_lvq

In [ ]:
plots.model_evaluate(lvq_model, df_lvq, X_test, Y_test, model_name="LVQ")